## First Import the necessary pkages




In [7]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 6.1 MB/s eta 0:00:00


In [25]:
import os
import requests
import json
from typing import List
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI #if you wanna use open ai
from groq import Groq #if you wanna use GroqCloud for free models like llama
from IPython.display import Markdown, display #This is for viasualising the markdown format

## Get the groq or open ai api key from Secrets

In [10]:
from google.colab import userdata
api_key = userdata.get('GROQ_API_KEY')
# userdata.get('OPEN_AI_API_KEY')

# Check the key
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [11]:
# constants for groq i am using lama 70b model
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama-3.3-70b-versatile'

## Create Chat Clients for the models

In [17]:
client = Groq(api_key=api_key)
# client = OpenAI(api_key=api_key)

#check if the client sends a response
response = client.chat.completions.create(
    model=MODEL_LLAMA,
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
    ],
)

print(response.choices[0].message.content)

The Los Angeles Dodgers won the 2020 World Series, defeating the Tampa Bay Rays in the series 4 games to 2. It was the Dodgers' first World Series title since 1988. The series was played from October 20 to October 27, 2020, at Globe Life Field in Arlington, Texas, due to the COVID-19 pandemic.


## Now using prompting style we will define User prompt and system prompt.
A **system prompt** tells the model what is it and how should it reply where **user prompt** means based on what query the model should reply.


In [21]:
system_prompt = """You are an assistant that analyzes code provided by a user and explain the code
line by line so that its very easy to understand for the user. Respond in markdown."""
user_prompt = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [22]:
#lets see it in action
response = client.chat.completions.create(
    model=MODEL_LLAMA,
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
)

print(response.choices[0].message.content)

### Explanation of the Code
```python
yield from {book.get("author") for book in books if book.get("author")}
```
Let's break down the code line by line:

#### 1. `yield from` Statement
*   The `yield from` statement is used in Python to yield items from an iterable.
*   It's a way to delegate part of the iteration to a sub-generator.

#### 2. `{...}` Set Comprehension
*   The `{...}` is a set comprehension, which is similar to a list comprehension but creates a set instead of a list.
*   Sets in Python are unordered collections of unique elements.

#### 3. `book.get("author") for book in books`
*   This is a loop within the set comprehension that iterates over each `book` in the `books` collection.
*   The `book.get("author")` call attempts to retrieve the value associated with the key `"author"` from each `book`.
*   The `.get()` method is used instead of directly accessing the key (`book["author"]`) to avoid a `KeyError` if the key is missing.
*   If the key is missing, `.get()` ret

In [26]:
display(Markdown(response.choices[0].message.content))

### Explanation of the Code
```python
yield from {book.get("author") for book in books if book.get("author")}
```
Let's break down the code line by line:

#### 1. `yield from` Statement
*   The `yield from` statement is used in Python to yield items from an iterable.
*   It's a way to delegate part of the iteration to a sub-generator.

#### 2. `{...}` Set Comprehension
*   The `{...}` is a set comprehension, which is similar to a list comprehension but creates a set instead of a list.
*   Sets in Python are unordered collections of unique elements.

#### 3. `book.get("author") for book in books`
*   This is a loop within the set comprehension that iterates over each `book` in the `books` collection.
*   The `book.get("author")` call attempts to retrieve the value associated with the key `"author"` from each `book`.
*   The `.get()` method is used instead of directly accessing the key (`book["author"]`) to avoid a `KeyError` if the key is missing.
*   If the key is missing, `.get()` returns `None` by default.

#### 4. `if book.get("author")`
*   This is a conditional statement within the set comprehension that filters out books that do not have an author.
*   If `book.get("author")` returns a truthy value (i.e., the author is present), the expression is included in the set; otherwise, it's skipped.

### Why This Code is Useful
This code is useful when you have a list of books and you want to extract a set of unique authors from these books. It:

*   Avoids duplicate authors by using a set.
*   Handles cases where a book might not have an author.
*   Uses a generator expression (`yield from`) to efficiently yield the authors one by one, which is beneficial for large collections of books.

Here's an example of how this code might be used within a function:
```python
def get_unique_authors(books):
    """Yield unique authors from a list of books."""
    yield from {book.get("author") for book in books if book.get("author")}

# Example usage
books = [
    {"title": "Book1", "author": "Author1"},
    {"title": "Book2", "author": "Author2"},
    {"title": "Book3"},
    {"title": "Book4", "author": "Author1"},
]

unique_authors = get_unique_authors(books)
for author in unique_authors:
    print(author)
```
This would output:
```
Author1
Author2
```

### Lets modularise the code

In [29]:
system_prompt = """You are an assistant that analyzes code provided by a user and explain the code
line by line so that its very easy to understand for the user. Respond in markdown."""

In [30]:
def user_prompt_generator(code_snippets: str) -> str:
    return f"""
    Please explain what this code does and why:
    {code_snippets}
    """

In [32]:
def messages_for(code_snippets: str) -> str:
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_generator(code_snippets)}
    ]

In [37]:
def code_analyzer(MODEL: str, code_snippets: str ) -> str:
    response = client.chat.completions.create(
        model=MODEL,
        messages = messages_for(code_snippets)
    )
    bot_response =  response.choices[0].message.content
    display(Markdown(bot_response))

In [45]:
# this will reply as chat gpt stream output
def code_analyzer_stream_output(MODEL: str, code_snippets: str ) -> str:
    stream = client.chat.completions.create(
        model=MODEL,
        messages = messages_for(code_snippets),
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [39]:
userInput = input("Enter the code snippet you want to analyze: ")
code_analyzer(MODEL_LLAMA, userInput)

Enter the code snippet you want to analyze: headers = {  "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36" }  class Website:      def __init__(self, url):         """         Create this Website object from the given url using the BeautifulSoup library         """         self.url = url         response = requests.get(url, headers=headers)         soup = BeautifulSoup(response.content, 'html.parser')         self.title = soup.title.string if soup.title else "No title found"         for irrelevant in soup.body(["script", "style", "img", "input"]):             irrelevant.decompose()         self.text = soup.body.get_text(separator="\n", strip=True)


### Code Explanation
#### Importing Libraries and Setting User Agent
```python
headers = {  "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36" }
```
This line sets a dictionary `headers` with a `"User-Agent"` key. The value of this key is a string that identifies the browser and operating system of the client making the HTTP request. This is done to mimic the behavior of a real web browser and avoid being blocked by websites that don't want to be scraped.

#### Defining the Website Class
```python
class Website:
```
This line defines a new class called `Website`.

#### Initializing the Website Object
```python
def __init__(self, url):
```
This is the constructor method of the `Website` class, which is called when an object of this class is instantiated. The `url` parameter is the URL of the website that the object will represent.

#### Docstring for the Constructor Method
```python
"""Create this Website object from the given url using the BeautifulSoup library"""
```
This is a docstring that provides a description of the constructor method. It explains that the method creates a `Website` object from a given URL using the BeautifulSoup library.

#### Sending an HTTP Request and Getting the Response
```python
self.url = url
response = requests.get(url, headers=headers)
```
Here, the `url` attribute of the `Website` object is set to the provided `url`. Then, an HTTP GET request is sent to the specified `url` with the previously defined `headers`. The response from the server is stored in the `response` variable.

#### Parsing the HTML Content
```python
soup = BeautifulSoup(response.content, 'html.parser')
```
The HTML content of the response is parsed using the `BeautifulSoup` library, which creates a parse tree that can be used to extract data from the HTML.

#### Extracting the Title of the Website
```python
self.title = soup.title.string if soup.title else "No title found"
```
This line extracts the title of the website from the parse tree. If the website has a title, it is stored in the `title` attribute of the `Website` object. If no title is found, the string "No title found" is stored instead.

#### Removing Irrelevant Elements from the Parse Tree
```python
for irrelevant in soup.body(["script", "style", "img", "input"]):
    irrelevant.decompose()
```
This loop goes through all the elements in the body of the HTML that are of type `script`, `style`, `img`, or `input`. These elements are considered irrelevant because they do not contain text that is visible to the user. The `decompose` method is used to remove these elements from the parse tree.

#### Extracting the Text from the Parse Tree
```python
self.text = soup.body.get_text(separator="\n", strip=True)
```
Finally, this line extracts the text from the parse tree, excluding any irrelevant elements that were removed in the previous step. The `separator` parameter is set to `\n` to separate the text with newline characters, and the `strip` parameter is set to `True` to remove any leading or trailing whitespace from the text. The extracted text is stored in the `text` attribute of the `Website` object.

### Why This Code is Useful
This code is useful for scraping websites and extracting relevant information from them. By removing irrelevant elements like scripts, styles, images, and input fields, it makes it easier to extract the actual text content of a website. This can be useful for tasks like text analysis, sentiment analysis, or data mining. Additionally, by setting a User-Agent header, the code can avoid being blocked by websites that don't want to be scraped.

In [43]:
userInput = input("Enter the code snippet you want to analyze: ")
code_analyzer_stream_output(MODEL_LLAMA, userInput)

Enter the code snippet you want to analyze: def stream_brochure(company_name, url):     stream = Groqclient.chat.completions.create(         model=MODEL,         messages=[             {"role": "system", "content": system_prompt},             {"role": "user", "content": get_brochure_user_prompt(company_name, url)}           ],         stream=True     )          response = ""     display_handle = display(Markdown(""), display_id=True)     for chunk in stream:         response += chunk.choices[0].delta.content or ''         response = response.replace("```","").replace("markdown", "")         update_display(Markdown(response), display_id=display_handle.display_id)


### Code Explanation
#### Function Definition
python
def stream_brochure(company_name, url):

This line defines a function named `stream_brochure` that takes two parameters: `company_name` and `url`.

#### Initialization of Stream
python
stream = Groqclient.chat.completions.create(

Here, the function `stream_brochure` is using the `Groqclient` library to create a stream of completions. The `Groqclient` library is likely an API client for a language model or chat service.

#### Stream Configuration
python
model=MODEL,
messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
],
stream=True

In this section, the function is configuring the stream with the following settings:
* `model`: The language model to use for generating completions. The `MODEL` variable is not defined in this snippet, but it's likely a pre-trained model.
* `messages`: A list of messages to use as input for the language model. The list contains two messages:
	+ The first message is a system prompt, which is a predefined string stored in the `system_prompt` variable.
	+ The second message is a user prompt generated by the `get_brochure_user_prompt` function, which takes `company_name` and `url` as input.
* `stream=True`: This setting enables streaming mode, which allows the function to receive a continuous stream of completions from the language model.

#### Response Initialization
python
response = ""
display_handle = display(Markdown(""), display_id=True)

Here, the function initializes an empty string `response` to store the generated completion. It also creates a display handle using the `display` function from a library like Jupyter Notebook or a similar environment. The display handle is used to update the output in real-time.

#### Streaming Completions
python
for chunk in stream:
    response += chunk.choices[0].delta.content or ''

This loop iterates over the stream of completions generated by the language model. For each chunk, it extracts the content of the first choice (i.e., the most likely completion) and appends it to the `response` string.

#### Response Processing
python
response = response.replace("","").replace("", "")

Here, the function processes the `response` string by removing backticks (`` ` ``) and the string "".

#### Updating Display
python
update_display(Markdown(response), display_id=display_handle.display_id)

Finally, the function updates the display using the `update_display` function, passing the processed `response` string as Markdown content and the display ID from the display handle. This updates the output in real-time, allowing the user to see the generated completion as it is being generated.

### Why This Code Exists
This code is likely part of a larger application that uses a language model to generate content, such as a brochure, based on a company name and URL. The `stream_brochure` function is designed to generate this content in real-time, using a streaming API to receive a continuous stream of completions from the language model. The function processes the generated content, removes unwanted characters, and updates the display in real-time, allowing the user to see the generated brochure as it is being generated.